In [1]:
%matplotlib inline
import itertools
import os
os.environ['CUDA_VISIBLE_DEVICES']=""
import numpy as np
import gpflow
import gpflow.training.monitor as mon
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf

# Demo: `gpflow.training.monitor`
In this notebook we'll demo how to use `gpflow.training.monitor` for logging the optimisation of a GPflow model.

## Creating the GPflow model
We first generate some random data and create a GPflow model.

Under the hood, GPflow gives a unique name to each model which is used to name the Variables it creates in the TensorFlow graph containing a random identifier. This is useful in interactive sessions, where people may create a few models, to prevent variables with the same name conflicting. However, when loading the model, we need to make sure that the names of all the variables are exactly the same as in the checkpoint. This is why we pass name="SVGP" to the model constructor, and why we use gpflow.defer_build().

In [2]:
np.random.seed(0)
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(Xt) + np.random.randn(*Xt.shape)

with gpflow.defer_build():
    m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(),
                           Z=np.linspace(0, 10, 5)[:, None],
                           minibatch_size=100, name="SVGP")
    m.likelihood.variance = 0.01
m.compile()

Let's compute log likelihood before the optimisation

In [3]:
print('LML before the optimisation: %f' % m.compute_log_likelihood())

LML before the optimisation: -1271605.621944


We will be using a TensorFlow optimiser. All TensorFlow optimisers have a support for `global_step` variable. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

The code below creates this variable using a monitor's helper function. It is important to create it before building the monitor in case the monitor includes a checkpoint task. This is because the checkpoint internally uses the TensorFlow Saver which creates a list of variables to save. Therefore all variables expected to be saved by the checkpoint task should exist by the time the task is created.

In [4]:
session = m.enquire_session()
global_step = mon.create_global_step(session)

## Construct the monitor

Next we need to construct the monitor. `gpflow.training.monitor` provides classes that are building blocks for the monitor. Essengially, a monitor is a function that is provided as a callback to an optimiser. It consists of a number of tasks that may be executed at each step, subject to their running condition.

In this example, we want to:
- log certain scalar parameters in TensorBoard,
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches,
- store a backup of the optimisation process periodically,
- log performance for a test set periodically.

We will define these tasks as follows:

In [6]:
print_task = mon.PrintTimingsTask().with_name('print')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

sleep_task = mon.SleepTask(0.01).with_name('sleep').with_name('sleep')

saver_task = mon.CheckpointTask('./monitor-saves').with_name('saver')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

model_tboard_task = mon.ModelToTensorBoardTask('./model-tensorboard', m).with_name('model_tboard')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

lml_tboard_task = mon.LmlToTensorBoardTask('./model-tensorboard', m).with_name('lml_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

As the above code shows, each task can be assigned a name and running conditions. The name will be shown in the task timing summary.

There are two different types of running conditions: `with_condition` controls execution of the task at each iteration in the optimisation loop. `with_exit_condition` is a simple boolean flag indicating that the task should also run at the end of optimisation.
In this example we want to run our tasks periodically, at every iteration or every 10th or 100th iteration.


## Custom tasks
We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, we may want to compute the performance on a test set. Here we create such a class by extending `BaseTensorBoardTask` to log the testing benchmarks in addition to all the scalar parameters.

In [7]:
class CustomTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, event_path, model, Xt, Yt):
        super().__init__(event_path, model)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:
        minibatch_size = 100
        preds = np.vstack([self.model.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        self._eval_summary(context, {self._full_test_err: test_err, self._full_test_nlpp: 0.0})

        
custom_tboard_task = CustomTensorBoardTask('./model-tensorboard', m, Xt, Yt).with_name('custom_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

Now we can put all these tasks into a monitor.

In [8]:
monitor_tasks = [print_task, model_tboard_task, lml_tboard_task, custom_tboard_task, saver_task, sleep_task]
monitor = mon.Monitor(monitor_tasks, session, global_step)

## Running the optimisation
We finally get to running the optimisation.

We may want to continue a previously run optimisation by resotring the TensorFlow graph from the latest checkpoint. Otherwise skip this step.

In [9]:
if os.path.isdir('./monitor-saves'):
    mon.restore_session(session, './monitor-saves')

In [10]:
try:
    optimiser = gpflow.train.AdamOptimizer(0.01)
    monitor.start_monitoring()
    optimiser.minimize(m, step_callback=monitor, maxiter=450, global_step=global_step)
finally:
    monitor.stop_monitoring()
    monitor.print_summary()

Iteration 10	total itr.rate 13.16/s	recent itr.rate 13.16/s	opt.step 10	total opt.rate 14.94/s	recent opt.rate 14.94/s
Iteration 20	total itr.rate 19.61/s	recent itr.rate 38.46/s	opt.step 20	total opt.rate 29.12/s	recent opt.rate 575.34/s
Iteration 30	total itr.rate 24.39/s	recent itr.rate 47.60/s	opt.step 30	total opt.rate 42.83/s	recent opt.rate 724.09/s
Iteration 40	total itr.rate 27.88/s	recent itr.rate 48.92/s	opt.step 40	total opt.rate 55.99/s	recent opt.rate 721.23/s
Iteration 50	total itr.rate 29.43/s	recent itr.rate 37.83/s	opt.step 50	total opt.rate 68.82/s	recent opt.rate 825.46/s
Iteration 60	total itr.rate 31.45/s	recent itr.rate 47.94/s	opt.step 60	total opt.rate 81.09/s	recent opt.rate 744.45/s
Iteration 70	total itr.rate 33.19/s	recent itr.rate 49.69/s	opt.step 70	total opt.rate 92.86/s	recent opt.rate 721.66/s
Iteration 80	total itr.rate 34.63/s	recent itr.rate 49.66/s	opt.step 80	total opt.rate 104.43/s	recent opt.rate 816.60/s
Iteration 90	total itr.rate 35.74/s	rece

 30%|███       | 30/100 [00:00<00:00, 298.94it/s]

Iteration 100	total itr.rate 36.64/s	recent itr.rate 47.38/s	opt.step 100	total opt.rate 124.19/s	recent opt.rate 474.11/s


100%|██████████| 100/100 [00:00<00:00, 400.91it/s]


Iteration 110	total itr.rate 31.42/s	recent itr.rate 12.95/s	opt.step 110	total opt.rate 133.61/s	recent opt.rate 553.75/s
Iteration 120	total itr.rate 32.27/s	recent itr.rate 45.97/s	opt.step 120	total opt.rate 142.60/s	recent opt.rate 548.34/s
Iteration 130	total itr.rate 33.00/s	recent itr.rate 45.43/s	opt.step 130	total opt.rate 151.31/s	recent opt.rate 566.96/s
Iteration 140	total itr.rate 33.76/s	recent itr.rate 48.11/s	opt.step 140	total opt.rate 161.15/s	recent opt.rate 1038.98/s
Iteration 150	total itr.rate 34.62/s	recent itr.rate 53.92/s	opt.step 150	total opt.rate 170.67/s	recent opt.rate 989.90/s
Iteration 160	total itr.rate 35.34/s	recent itr.rate 51.25/s	opt.step 160	total opt.rate 179.62/s	recent opt.rate 840.40/s
Iteration 170	total itr.rate 35.90/s	recent itr.rate 48.23/s	opt.step 170	total opt.rate 188.08/s	recent opt.rate 764.97/s
Iteration 180	total itr.rate 36.40/s	recent itr.rate 47.51/s	opt.step 180	total opt.rate 196.34/s	recent opt.rate 774.12/s
Iteration 190	t

...

Iteration 410	total itr.rate 34.89/s	recent itr.rate 13.85/s	opt.step 410	total opt.rate 320.14/s	recent opt.rate 484.67/s
Iteration 420	total itr.rate 35.10/s	recent itr.rate 46.96/s	opt.step 420	total opt.rate 323.51/s	recent opt.rate 569.15/s
Iteration 430	total itr.rate 35.33/s	recent itr.rate 48.21/s	opt.step 430	total opt.rate 327.65/s	recent opt.rate 707.55/s
Iteration 440	total itr.rate 35.52/s	recent itr.rate 46.71/s	opt.step 440	total opt.rate 330.48/s	recent opt.rate 526.33/s
Iteration 450	total itr.rate 35.71/s	recent itr.rate 46.68/s	opt.step 450	total opt.rate 334.60/s	recent opt.rate 739.85/s


 48%|████▊     | 48/100 [00:00<00:00, 477.85it/s]

Iteration 450	total itr.rate 35.04/s	recent itr.rate 0.00/s	opt.step 450	total opt.rate 303.81/s	recent opt.rate 0.00/s


100%|██████████| 100/100 [00:00<00:00, 473.82it/s]


Tasks execution time summary:
print:	0.0384 (sec)
model_tboard:	0.1869 (sec)
lml_tboard:	1.6498 (sec)
custom_tboard:	1.3901 (sec)
saver:	4.0261 (sec)
sleep:	4.5390 (sec)


Now lets compute the log likelihood again. Hopefully we will see an increase in its value

In [11]:
print('LML after the optimisation: %f' % m.compute_log_likelihood())

LML after the optimisation: -68705.124191
